## Imports

In [1]:
%load_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
import linearmodels as lm

import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
import sys
sys.path.append('../')
sys.path.append('../../kungfu/')
import kungfu as kf
import src

/home/fbrun/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## Settings

In [4]:
pd.set_option('display.float_format', lambda x: '%.4f' % x) #sets decimals to be displayed

In [5]:
#sns.set()
sns.set(rc={'figure.figsize': (17, 6)})

In [6]:
# plt.style.use('fivethirtyeight')
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [17, 8]
#plt.rcParams['figure.dpi'] = 80

## Load estimates

In [75]:
# #
# estimation_stats = pd.read_csv('../data/estimated/estimation_statistics.csv')
# estimation_stats.index = pd.to_datetime(estimation_stats.iloc[:, 0].values)
# estimation_stats = estimation_stats.iloc[:,1:]

#
network_data = pd.read_csv('../data/estimated/network_data.csv')
network_data.sampling_date = pd.to_datetime(network_data.sampling_date)
network_data.permno = network_data.permno.astype('int').astype('str')
network_data = network_data.set_index(['sampling_date', 'permno'])

#
df_stats = pd.read_csv('../data/estimated/summary_stats.csv')
df_stats.sampling_date = pd.to_datetime(df_stats.sampling_date)
df_stats.permno = df_stats.permno.astype('str')
df_stats = df_stats.set_index(['sampling_date', 'permno'])#drop(columns=['sampling_date'])

#
df_factor = pd.read_csv('../data/estimated/factor_exposures.csv')
df_factor.sampling_date = pd.to_datetime(df_factor.sampling_date)
df_factor.permno = df_factor.permno.astype('str')
df_factor = df_factor.set_index(['sampling_date', 'permno'])#drop(columns=['sampling_date'])

# #
# df_spy = pd.read_csv('../data/estimated/spy_stats.csv')
# df_spy.index = pd.to_datetime(df_spy.iloc[:, 0].values)
# df_spy = df_spy.iloc[:,1:]

In [100]:
df_merged = network_data.merge(df_stats, left_index=True, right_on=['sampling_date', 'permno'])\
                        .merge(df_factor, left_index=True, right_on=['sampling_date', 'permno'])
df_merged.index = pd.MultiIndex.from_arrays([df_merged.index.get_level_values('permno').astype('int'),
                                             df_merged.index.get_level_values('sampling_date')])

In [137]:
df_merged.columns.to_list()

['var_intercept',
 'mean_abs_var_in',
 'mean_abs_var_out',
 'residual_variance',
 'mean_resid_corr',
 'fev_total',
 'fev_others',
 'fev_self',
 'fev_in_connectedness',
 'fev_out_connectedness',
 'fev_eigenvector_centrality',
 'fev_closeness_centrality',
 'fev_in_entropy',
 'fev_in_connectedness_normalised',
 'fev_out_connectedness_normalised',
 'fev_eigenvector_centrality_normalised',
 'fev_closeness_centrality_normalised',
 'fu_total',
 'fu_others',
 'fu_self',
 'fu_in_connectedness',
 'fu_out_connectedness',
 'fu_eigenvector_centrality',
 'fu_closeness_centrality',
 'fu_in_entropy',
 'fu_in_connectedness_normalised',
 'fu_out_connectedness_normalised',
 'fu_eigenvector_centrality_normalised',
 'fu_closeness_centrality_normalised',
 'ret_back_12M',
 'var_back_12M',
 'ret_forward_1M',
 'var_forward_1M',
 'ret_forward_2M',
 'var_forward_2M',
 'ret_forward_3M',
 'var_forward_3M',
 'ret_forward_4M',
 'var_forward_4M',
 'ret_forward_5M',
 'var_forward_5M',
 'ret_forward_6M',
 'var_forward_

In [140]:
lm.PanelOLS(dependent=df_merged['var_forward_12M'],
            exog=df_merged[['fev_eigenvector_centrality', 'var_back_12M']],
            time_effects=True,
            entity_effects=False,
           )\
    .fit(cov_type='kernel')

Dep. Variable:,var_forward_12M,R-squared:,0.0920
Estimator:,PanelOLS,R-squared (Between):,0.6488
No. Observations:,31197,R-squared (Within):,0.0121
Date:,"Tue, Feb 23 2021",R-squared (Overall):,0.2354
Time:,13:51:02,Log-likelihood,-1596.2
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,1563.9
Entities:,333,P-value,0.0000
Avg Obs:,93.685,Distribution:,"F(2,30883)"
Min Obs:,1.0000,,
Max Obs:,312.00,F-statistic (robust):,8.1420


In [146]:
lm.PanelOLS(dependent=df_merged['var_forward_6M'],
            exog=df_merged[['fev_eigenvector_centrality', 'var_back_12M']],
            time_effects=True,
            entity_effects=False,
           )\
    .fit(cov_type='kernel')

Dep. Variable:,var_forward_6M,R-squared:,0.1336
Estimator:,PanelOLS,R-squared (Between):,0.6871
No. Observations:,31797,R-squared (Within):,0.0406
Date:,"Tue, Feb 23 2021",R-squared (Overall):,0.2890
Time:,13:52:16,Log-likelihood,3107.8
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,2426.7
Entities:,338,P-value,0.0000
Avg Obs:,94.074,Distribution:,"F(2,31477)"
Min Obs:,1.0000,,
Max Obs:,318.00,F-statistic (robust):,7.2707
